## Setting Up

### Install dependencies

In [2]:
!pip install -U tensorflow==2.3.0
!pip install -U numpy==1.18.5
!pip install -U PILLOW==7.2.0
!pip install -U opencv-python==4.5.1.48

  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datature-hub 0.2.1 requires opencv-python==4.5.1.48, but you have opencv-python 4.5.5.64 which is incompatible.
datature-hub 0.2.1 requires Pillow~=8.2, but you have pillow 9.1.1 which is incompatible.
You should consider upgrading via the 'C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datature-hub 0.2.1 requires opencv-python==4.5.1.48, but you have opencv-python 4.5.5.64 which is incompatible.
datature-hub 0.2.1 requires Pillow~=8.2, but you have pillow 7.2.0 which is incompatible.
You should consider upgrading via the 'C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.



  Using cached Pillow-7.2.0-cp38-cp38-win_amd64.whl (2.1 MB)
  Attempting uninstall: PILLOW
    Found existing installation: Pillow 9.1.1
    Uninstalling Pillow-9.1.1:
      Successfully uninstalled Pillow-9.1.1
  Using cached opencv_python-4.5.1.48-cp38-cp38-win_amd64.whl (34.9 MB)
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.5.5.64
    Uninstalling opencv-python-4.5.5.64:
      Successfully uninstalled opencv-python-4.5.5.64


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datature-hub 0.2.1 requires Pillow~=8.2, but you have pillow 7.2.0 which is incompatible.
You should consider upgrading via the 'C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


### Imports

In [1]:
import os
import cv2
import time
import glob
import argparse
import numpy as np
import tensorflow as tf

from PIL import Image

os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # Comment out to use GPU for prediction

## Prediction Preparation

### Set all necessary variables

In [9]:
input_folder = "./input" # Path to folder that contains input images
output_folder = "./output" # Path to folder to store predicted images
width = 640 # Width of image to load into model
height = 640 # Height of image to load into model
threshold = 0.7 # Prediction confidence threshold
model = "./saved_model" # Path to exported tensorflow pb model
label = "./label_map.pbtxt" # Path to label map

### Label map loader

In [10]:
def load_label_map(label_map_path):
    label_map = {}

    with open(label_map_path, "r") as label_file:
        for line in label_file:
            if "id" in line:
                label_index = int(line.split(":")[-1])
                label_name = next(label_file).split(":")[-1].strip().strip("'")
                label_map[label_index] = {"id": label_index, "name": label_name}

    return label_map

### Image to numpy array 

In [11]:
def load_image_into_numpy_array(path, height, width):
    image = Image.open(path).convert("RGB")
    image_shape = np.asarray(image).shape
    image_resized = image.resize((height, width))
    
    return np.array(image_resized), (image_shape[0], image_shape[1])

## Execute Prediction 

### Load label_map, color_map and model 

In [12]:
category_index = load_label_map(label)

color_map = {}
for each_class in range(len(category_index)):
    color_map[each_class] = [int(i) for i in np.random.choice(range(256), size=3)]

start_time = time.time()
trained_model = tf.saved_model.load(model)
print("Model loaded, took {} seconds...".format(time.time() - start_time))

NameError: name 'tf' is not defined

### Loop through images and obtain predictions

In [13]:
## Run prediction on each image
for each_image in glob.glob(os.path.join(input_folder, "*")):
    print("Predicting for {}...".format(each_image))

    ## Returned original_shape is in the format of width, height
    image_resized, origi_shape = load_image_into_numpy_array(
        each_image, int(height), int(width)
    )

    ## The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_resized)

    ## The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    ## Feed image into model
    detections_output = trained_model(input_tensor)

    num_detections = int(detections_output.pop("num_detections"))
    detections = {
        key: value[0, :num_detections].numpy()
        for key, value in detections_output.items()
    }
    detections["num_detections"] = num_detections

    ## Filter out predictions below threshold
    indexes = np.where(detections["detection_scores"] > float(threshold))

    ## Extract predictions
    bboxes = detections["detection_boxes"][indexes]
    classes = detections["detection_classes"][indexes].astype(np.int64)
    scores = detections["detection_scores"][indexes]

    ## Draw Predictions
    image_origi = Image.fromarray(image_resized).resize(
        (origi_shape[1], origi_shape[0])
    )
    image_origi = np.array(image_origi)

    if len(bboxes) != 0:
        for idx, each_bbox in enumerate(bboxes):
            color = color_map.get(classes[idx] - 1)

            ## Draw bounding box
            cv2.rectangle(
                image_origi,
                (
                    int(each_bbox[1] * origi_shape[1]),
                    int(each_bbox[0] * origi_shape[0]),
                ),
                (
                    int(each_bbox[3] * origi_shape[1]),
                    int(each_bbox[2] * origi_shape[0]),
                ),
                color,
                2,
            )

            ## Draw label background
            cv2.rectangle(
                image_origi,
                (
                    int(each_bbox[1] * origi_shape[1]),
                    int(each_bbox[2] * origi_shape[0]),
                ),
                (
                    int(each_bbox[3] * origi_shape[1]),
                    int(each_bbox[2] * origi_shape[0] + 15),
                ),
                color,
                -1,
            )

            ## Insert label class & score
            cv2.putText(
                image_origi,
                "Class: {}, Score: {}".format(
                    str(category_index[classes[idx]]["name"]),
                    str(round(scores[idx], 2)),
                ),
                (
                    int(each_bbox[1] * origi_shape[1]),
                    int(each_bbox[2] * origi_shape[0] + 10),
                ),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.3,
                (0, 0, 0),
                1,
                cv2.LINE_AA,
            )

    ## Save predicted image
    filename = os.path.basename(each_image)
    image_predict = Image.fromarray(image_origi)
    image_predict.save(os.path.join(output_folder, filename))

    print(
        "Saving predicted images to {}...".format(
            os.path.join(output_folder, filename)
        )
    )

Predicting for ./input\361.jpg...


NameError: name 'Image' is not defined